### Hyperparameter Tuning for DeepCT-Query Reduction

In [2]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded
ensure_pyterrier_is_loaded()

import pyterrier as pt
import os
import sys
import pandas as pd
from tqdm import tqdm
sys.path.append('deepct-query-reduction')
from deepct_query_reduction import DeepCTQueryReduction


index = pt.IndexRef.of(os.path.abspath('pyterrier-index/index/data.properties'))

In [3]:
MODELS = [ "/deepct-models/deepct-main-01/output/model.ckpt-20000",
          #"/deepct-models/deepct-main-01/output/model.ckpt-22503", 
          #"/deepct-models/deepct-title-01/output/model.ckpt-20000",
          #"/deepct-models/deepct-title-01/output/model.ckpt-21166", "/deepct-models/deepct-main-01/output/model.ckpt-0",
          #"/deepct-models/deepct-main-02/output/model.ckpt-0", "/deepct-models/deepct-title-01/output/model.ckpt-0",
]
OMIT_DUPLICATES = ['True', 'False']
PROPORTIONS = ['0.05', '0.10', '0.15', '0.20', '0.25', '0.30', '0.35', '0.40', '0.45', '0.50', '0.55', '0.60', '0.65', '0.70', '0.75', '0.80', '0.85', '0.90', '0.95', '1.00']
pipelines = []

for model in MODELS:
    for omit in OMIT_DUPLICATES:
        for proportion in PROPORTIONS:
            pipelines += [model + ';' + proportion + ';' + omit]

print(len(pipelines), 'Pipelines')

40 Pipelines


In [4]:
def report_effectiveness(pipeline, dataset):
    queries = pt.io.read_topics(f'tot-{dataset}/queries.xml', 'trecxml')
    deep_ct_query_reduction = DeepCTQueryReduction(f'deepct-query-reduction/tot-{dataset}-deepct-predictions/predictions.json', pipeline).as_transformer()

    bm25 = pt.BatchRetrieve(index, wmodel="BM25")

    retrieval_pipeline = deep_ct_query_reduction >> bm25

    ret = pt.Experiment([retrieval_pipeline], queries, pt.io.read_qrels(f'tot-{dataset}/qrels.txt'), eval_metrics=["ndcg_cut_10", 'recip_rank', 'P_10'])
    
    return {i: ret.iloc[0][i] for i in ['ndcg_cut_10', 'recip_rank', 'P_10']}

def report_pipeline(pipeline):
    ret = {'Pipeline': pipeline}
    for dataset in ['train', 'dev']:
        for k,v in report_effectiveness(pipeline, dataset).items():
            ret[dataset + '_' + k] = v
    return ret

df = []
for pipeline in tqdm(pipelines):
    df += [report_pipeline(pipeline)]

df = pd.DataFrame(df)
df.to_json('hyperparameter-tuning-deepct-results-variant-01.jsonl', lines=True, orient='records')

  0%|          | 0/40 [00:00<?, ?it/s]

In [7]:
pd.set_option('display.max_colwidth', None)
df

,Pipeline,train_ndcg_cut_10,train_recip_rank,train_P_10,dev_ndcg_cut_10,dev_recip_rank,dev_P_10
0,/deepct-models/deepct-main-01/output/model.ckpt-22503;0.25;True,0.041535,0.038080,0.006667,0.053785,0.048123,0.008667
1,/deepct-models/deepct-main-01/output/model.ckpt-22503;0.50;True,0.088932,0.080261,0.013333,0.094506,0.091035,0.012667
2,/deepct-models/deepct-main-01/output/model.ckpt-22503;0.75;True,0.096249,0.089650,0.013333,0.095243,0.091474,0.012667
3,/deepct-models/deepct-main-01/output/model.ckpt-22503;1.0;True,0.095325,0.088660,0.013333,0.091371,0.088743,0.012000
